# Inference code for reranker

In [ ]:
import sys
import torch

In [ ]:
sys.path.append("/private/home/samuelbroscheit/PycharmProjects/side-internal/")

In [ ]:
from misc.inference_notebook import get_crossencoder_components_from_checkpoint

In [ ]:
# The checkpoint
checkpoint_dir = "/checkpoint/fabiopetroni/WAI/Samuel/checkpoints/colbert/49816313/"

In [ ]:
ctx_src, biencoder_prepare_encoder_inputs_func, biencoder = \
    get_crossencoder_components_from_checkpoint([
    "retriever=reranker",
    "retriever.ctx_src=wafer_ccnet",
    "retriever.batch_size=2",
    "retriever.checkpoint_dir='/checkpoint/fabiopetroni/WAI/Samuel/checkpoints/crossencoder/52237054/'",
])

In [ ]:
def get_score(wiki_claim, passage):
    question_ids = ctx_src.preprocess_query(wiki_claim, training=False)
    ctx_ids = ctx_src.preprocess_passage(ctx_src.passage_struct_from_dict(passage), training=False)
    inputs = {
        "question_ids": question_ids.view(1, -1),
        "question_segments": torch.zeros_like(question_ids),
        "context_ids": ctx_ids.view(1, -1),
        "ctx_segments": torch.zeros_like(ctx_ids),
    }
    inputs = biencoder_prepare_encoder_inputs_func(inputs, ctx_src)
    with torch.no_grad():
        _, score = biencoder(
            **inputs,
            encoder_type="question",
            representation_token_pos=0,
        )
    return score
    

# Playground

In [ ]:
get_score(
    "U.K.[SEP]Some Section[SEP]The U.K. is an island.[CIT]",
    {
        "title": "Apparently the U.K. is an island! The Guardian",
        "text": "I was drinking my morning coffe when I realized that the U.K. is an island.",
        "url": "the guardian",
    },
)    